# Analysis of one-year trace of gut microbiome
This notebook records the code used for analyzing data from [Gibbons _et. al._ (2017)](http://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1005364). 

## Load required packages

In [1]:
library(beem)
library(grid)
library(ggplot2)
library(ggsci)
library(igraph)
library(reshape2)

Loading required package: foreach
Loading required package: doMC
Loading required package: iterators
Loading required package: parallel

Attaching package: ‘igraph’

The following objects are masked from ‘package:stats’:

    decompose, spectrum

The following object is masked from ‘package:base’:

    union



## Load functions and data

In [2]:
input.da <- read.table('~/BEEM/vignettes/gibbons_et_al_analysis/DA.counts.txt', head=F, row.names=1)
metadata.da <- read.table('~/BEEM/vignettes//gibbons_et_al_analysis/DA.metadata.txt', head=T)

## For DB, point #74 has extremely high of one species and #180 is sampled too far from the previous time point
input.db <- read.table('~/BEEM/vignettes//gibbons_et_al_analysis/DB.counts.txt', head=F, row.names=1)[,-c(74,180)]
metadata.db <- read.table('~/BEEM/vignettes/gibbons_et_al_analysis/DB.metadata.txt', head=T)[-c(74,180),]

## For M3, data from 330:332 are too far from previous time point
input.m3 <- read.table('~/BEEM/vignettes/gibbons_et_al_analysis/M3.counts.txt', head=F, row.names=1)[,1:329] 
metadata.m3 <- read.table('~/BEEM/vignettes/gibbons_et_al_analysis/M3.metadata.txt', head=T)[1:329,]

input.f4 <- read.table('~/BEEM/vignettes/gibbons_et_al_analysis/F4.counts.txt', head=F, row.names=1) 
metadata.f4 <- read.table('~/BEEM/vignettes/gibbons_et_al_analysis/F4.metadata.txt', head=T)

## Run BEEM

### Individual DA

In [ ]:
counts.da <- input.da[-1,]
colnames(counts.da) <- as.character(input.da[1,])
res.da <- EM(dat=counts.da,meta=metadata.da, dev=10, verbose=FALSE,
          min_iter=50, max_iter=100, epsilon = 0.5, 
          scaling = 10000, ncpu=4, seed=0)

Warning message in EM(dat = counts.da, meta = metadata.da, dev = 10, verbose = FALSE, :
“Small number (<10) of biological replicates detected. Note that BEEM works best with >10 biological replicates or the time series contains intrinsic infrequent perturbations.”The following species are not recommended due to 0 values:

The following species are not recommended due to their low/high abudances:
552961, 682726, 529021, 188634, 181765, 593352, 302439, 574965, 1809696, 4407174, 4459708, 310391, 177224, 1073276, 586030, 1106789, 851715, 328438, 316489, 198583, 365868
The following species is recommended as the reference:
1111783
BEEM selecting reference species as default...
Reference species: 1111783


### Individual M3

In [ ]:
counts.m3 <- input.m3[-1,]
colnames(counts.m3) <- as.character(input.m3[1,])
res.m3 <- EM(dat=counts.m3, meta=metadata.m3,  dev=10, verbose=FALSE,
          min_iter=50, max_iter=100, epsilon = 0.5, 
          scaling = 10000, ncpu=4, seed=0)

### Individual DB

In [ ]:
counts.db <- input.db[-1,]
colnames(counts.db) <- as.character(input.db[1,])
res.db <- EM(dat=counts.db,meta=metadata.db, dev=10, verbose=FALSE,
          min_iter=50, max_iter=100, epsilon = 0.5, 
          scaling = 10000, ncpu=4, seed=0)

### Individual F4

In [ ]:
counts.f4 <- input.f4[-1,]
colnames(counts.f4) <- as.character(input.f4[1,])
res.f4 <- EM(dat=counts.f4,meta=metadata.f4, dev=10, verbose=FALSE,
          min_iter=50, max_iter=100, epsilon = 0.5, 
          scaling = 10000, ncpu=4, seed=0)

## Infer parameters

In [ ]:
params.da <- paramFromEM(res.da, counts.da, metadata.da, ncpu=4)
params.m3 <- paramFromEM(res.m3, counts.m3, metadata.m3, ncpu=4)
params.db <- paramFromEM(res.db, counts.db, metadata.db, ncpu=4)
params.f4 <- paramFromEM(res.f4, counts.f4, metadata.f4, ncpu=4)

## Functions for analysis

In [ ]:
getbiomass <- function(res){
    trace.mse <- res$trace.mse
    min.mse <- min(trace.mse)
    em.idx <- which((trace.mse-min.mse) < res$epsilon*min.mse)
    biomass <- apply(res$trace.biomass[,em.idx],1,median)
    biomass
}
int.net <- function(counts, parms, sig=1, title){
    ## plot interaction network
    minmax <- function(x) (x-min(x))/(max(x)-min(x))
    annote <- read.table('~/BEEM/vignettes/gibbons_et_al_analysis/all_otu_mapping.txt',head=F, row.names=1)
    counts.mean <- rowMeans(counts)
    int <- parms[parms$parameter_type=='interaction' & parms$source_taxon!=parms$target_taxon,]
    int.f <- int[int$significance>sig,2:4]
    g <- graph.data.frame(int.f[,1:2])
    V(g)$color <- annote[V(g)$name,]$V4
    V(g)$size <- log(counts.mean[V(g)$name]) +4
    E(g)$color <- ifelse(int.f$value>0,fill_cols[12],fill_cols[13])
    E(g)$lty <- ifelse(int.f$value>0,1,2)
    E(g)$width <- minmax(abs(int.f$value) ) * 2 + 0.5
    plot(g, main=title,asp=0,edge.arrow.size=0.5,edge.curved=.15)
    return(g)
}

## Biomass trajectory of individual DA
Note the periodic behaviour of the biomass -- the period is around 90 days (i.e. 3 months).

In [ ]:
par(mfrow = c(4,1))
plot(x=metadata.da$measurementID,y=getbiomass(res.da), xlim=c(0, 450), type='b', pch=19, xlab='Date', ylab='Estimated biomass', log='y')
plot(x=metadata.m3$measurementID,y=getbiomass(res.m3), xlim=c(0, 450), type='b', pch=19, xlab='Date', ylab='Estimated biomass', log='y')
plot(x=metadata.db$measurementID,y=getbiomass(res.db), xlim=c(0, 450), type='b', pch=19, xlab='Date', ylab='Estimated biomass', log='y')
plot(x=metadata.f4$measurementID,y=getbiomass(res.f4), xlim=c(0, 450), type='b', pch=19, xlab='Date', ylab='Estimated biomass', log='y')

## Plot interaction network

In [ ]:
fill_cols <- pal_simpsons(c("springfield"))(16)
ga <- int.net(counts.da, params.da, 1.5, 'DA')

In [ ]:
gm <- int.net(counts.m3, params.m3, 1.5, 'M3')

In [ ]:
gb <- int.net(counts.db, params.db, 1.5, 'DB')

In [ ]:
f4 <- int.net(counts.f4, params.f4, 1.5, 'F4')

In [ ]:
res.da$counts <- counts.da
res.da$metadata <- metadata.da
saveRDS(res.da, '~/BEEM/vignettes/gibbons_et_al_analysis/DA.EM.rds')
write.table(params.da, '~/BEEM/vignettes/gibbons_et_al_analysis/DA.params.txt', col.names=TRUE, row.names=FALSE, sep='\t', quote=FALSE)

res.m3$counts <- counts.m3
res.m3$metadata <- metadata.m3
saveRDS(res.m3, '~/BEEM/vignettes/gibbons_et_al_analysis/M3.EM.rds')
write.table(params.m3, '~/BEEM/vignettes/gibbons_et_al_analysis/M3.params.txt', col.names=TRUE, row.names=FALSE, sep='\t', quote=FALSE)

res.db$counts <- counts.db
res.db$metadata <- metadata.db
saveRDS(res.db, '~/BEEM/vignettes/gibbons_et_al_analysis/DB.EM.rds')
write.table(params.db, '~/BEEM/vignettes/gibbons_et_al_analysis/DB.params.txt', col.names=TRUE, row.names=FALSE, sep='\t', quote=FALSE)

res.f4$counts <- counts.f4
res.f4$metadata <- metadata.f4
saveRDS(res.f4, '~/BEEM/vignettes/gibbons_et_al_analysis/F4.EM.rds')
write.table(params.f4, '~/BEEM/vignettes/gibbons_et_al_analysis/F4.params.txt', col.names=TRUE, row.names=FALSE, sep='\t', quote=FALSE)